In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ccxt
import time
from datetime import datetime
import plotly.graph_objects as go
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import yfinance as yf
import plotly.express as px
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, plot,iplot
cf.go_offline()

In [2]:
cf.go_offline()
# we will build a function that will calculate RSI

key = 'ZT6j9cVi03zdNaVZaq8anCvIF86eH5vyJhAfh4YCFRCgRRra6zP297hOxfRX9Zwc'
secret = 'ABzCXOJ41tI0OAOKBUu8hPqDbLKP1JYxRc6jsYjgPmy5RTXF6q7IYYtbWalVkQnL'
client =  Client(key, secret,tld='us',testnet=True) #connect to testnet server

In [3]:
def get_data(symbol, start,intervals, end = None):
    """
    This function will generate time series data for any ticker specified

    Args:
        symbol (str): this will be the ticker symbol you want to look up
        start (str): start date
        end (str): end date
        interval (str): interval you want your data in 1m,5m,15m, 1hr, 1d...etc
    """
    client = Client()  # make an instance for the binance
    #condition if no end date is specified
    if end == None:
        end = 'now UTC'
    if intervals == '1m':
        intervals = Client.KLINE_INTERVAL_1MINUTE
    # get historical
    df =client.get_historical_klines(symbol, intervals, start_str=start,end_str=end)
    df = pd.DataFrame(df)  # convert it to a dataframe
    # name each column for this data frame
    df.columns = [
        "Date",
        "Open",
        "High",
        "Low",
        "Close",
        "Volume",
        "Close_time",
        "qav",
        "num_trades",
        "taker_base_vol",
        "taker_quote_vol",
        "ignore",
    ]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]
    df["Date"] = pd.to_datetime(df["Date"],unit='ms')
    for column in ["Open", "High", "Low", "Close", "Volume"]:
        df[column] = pd.to_numeric(df[column])
    return df

In [4]:
def RSI(data, time_period):
    """This function will take historical date and calculate the RSI score

    Args:
        data (data frame): historical data in the form of a pandas data frame
        time_period (_type_): specify trading period to calculate on 
    """
    # https://www.macroption.com/rsi-calculation/ -> rsi calculation
    #RSI = 100 - 100/(1+RS)
    
    #Step 1: calculate up and down moves
    data['Price_Diff'] = data['Close'].diff(1)
    
    data['Up_Moves'] = data['Price_Diff'].apply(lambda x: x if x > 0 else 0)
    data['Down_Moves'] = data['Price_Diff'].apply(lambda x: abs(x) if x < 0 else 0)
    data['Avg_Up'] = data['Up_Moves'].ewm(span=time_period).mean()
    data['Avg_Down'] = data['Down_Moves'].ewm(span=time_period).mean()
    data['RS'] = data['Avg_Up']/data['Avg_Down'] 
    data['RSI'] = data['RS'].apply(lambda x: 100-(100/(1+x)))
    #data=data.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    data = data.dropna()
    data = data.iloc[1: , :]
    return data

In [5]:
def RSI_Strategy(data,upper_limit,lower_limit):
    """Implement the RSI strategy
    buy - when the closing prices reaches above the upper threshold 
    sell - when the closing price reaches below the lower threshold

    Args:
        data (dataframe): historical data with rsi computed column
        upper_limit (numeric): upper threshold for over bought
        lower_limit (numeric): lower threshold for over sold

    Returns:
        dataframe: return updated data frame with buy/sell prices and signals 
    """
     #create strategy rules for RSI -> long when rsi 
    buy_price = []
    print(data)
    sell_price = []
    rsi_signal = []
    signal = 0
    entry = False
    rsi = [x for x in data['RSI']]
    prices = [x for x in data['Close']]
    print(len(rsi),len(prices))
    for i in range(len(rsi)):
        
        if rsi[i-1] > lower_limit and rsi[i] < lower_limit:
            if signal != 1 :
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                rsi_signal.append(signal)
                
               
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
                #entry = False
        elif rsi[i-1] < upper_limit and rsi[i] > upper_limit:
            if signal != -1 :
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                rsi_signal.append(signal)
               
              
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
                #entry == True
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            rsi_signal.append(0)
            #entry == False
   
            
            
    data['Buy Price'] = buy_price
    data['Sell Price'] = sell_price
    data['Signal'] = rsi_signal
    
    return data



In [6]:
stream_df = pd.DataFrame(columns=['start_time','first', 'high', 'low', 'close', 'volume', 'complete'])

In [7]:
def stream_candles(msg):
    
    
    stream_df = pd.DataFrame(columns=['start_time','first', 'high', 'low', 'close', 'volume', 'complete'])
    event_time = pd.to_datetime(msg['E'],unit='ms')
    start_time = pd.to_datetime(msg['k']['t'],unit='ms')
    first = float(msg['k']['o'])
    high = float(msg['k']['h'])
    low = float(msg['k']['l'])
    close = float(msg['k']['c'])
    volume = float(msg['k']['v'])
    complete = float(msg['k']['x'])
    
    print("Time: {} | Price: {}".format(event_time, close))
    stream_df = stream_df.append({'start_time': start_time,'first': first, 'high': high, 'low': low, 'close': close, 'volume': volume, 'complete': complete},ignore_index=True)

In [8]:
def trader(data,entered_trade=False):
    
    # start is required to initialise its internal loop
    twm.start()
    #stream_df = pd.DataFrame(columns=['start_time','first', 'high', 'low', 'close', 'volume', 'complete'])
    
    twm.start_kline_socket(callback = stream_candles, symbol = "BTCUSDT", interval = "1m")
    print(stream_df)

In [9]:
df = get_data('BTCUSDT', '5 March,2022','1h', end = None)
df = RSI(df, 14)
df = RSI_Strategy(df,70,30)
twm = ThreadedWebsocketManager()
trader(df,entered_trade=False)

                   Date      Open      High       Low     Close      Volume  \
2   2022-03-05 02:00:00  38987.25  39064.26  38827.77  38866.23  1073.56015   
3   2022-03-05 03:00:00  38866.24  39119.12  38831.16  39050.62  1008.60233   
4   2022-03-05 04:00:00  39050.61  39100.81  38407.59  38856.33  1829.01931   
5   2022-03-05 05:00:00  38856.32  39021.62  38801.40  39007.05  1057.04451   
6   2022-03-05 06:00:00  39007.06  39104.62  38936.61  38981.86  1078.28894   
..                  ...       ...       ...       ...       ...         ...   
189 2022-03-12 21:00:00  39128.08  39180.89  39062.00  39091.99   435.68537   
190 2022-03-12 22:00:00  39092.00  39169.69  39032.38  39101.98   373.32156   
191 2022-03-12 23:00:00  39101.98  39164.00  38730.67  38807.36  1031.15566   
192 2022-03-13 00:00:00  38807.35  38961.22  38703.73  38917.66  1106.43812   
193 2022-03-13 01:00:00  38917.65  39085.49  38883.19  39020.00   268.30896   

     Price_Diff  Up_Moves  Down_Moves      Avg_Up  

/tmp/ipykernel_10005/3402851740.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_10005/3402851740.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Time: 2022-03-13 01:10:02.024000 | Price: 39020.7


/tmp/ipykernel_10005/3402851740.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Time: 2022-03-13 01:10:04.025000 | Price: 39020.37


...

/tmp/ipykernel_10005/3402851740.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Time: 2022-03-13 01:10:14.334000 | Price: 39025.45


/tmp/ipykernel_10005/3402851740.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Time: 2022-03-13 01:10:17.743000 | Price: 39025.45


/tmp/ipykernel_10005/3402851740.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [10]:
twm.stop()

In [3]:
from datetime import timedelta

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [43]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
        # self.data = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days) # NEW!
            
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days): # NEW!
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df # Create self.data
    
    def stream_candles(self, msg):
        df2 = pd.DataFrame(columns = ["Date", "Open", "High", "Low", "Close", "Volume",	'Complete',	'Price_Diff',	'Up_Moves',	'Down_Moves',	'Avg_Up',	'Avg_Down'	,'RS'	,'RSI','Signal'])
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
        Price_Diff  = None
        Up_Moves  = None
        Down_Moves   = None  
        Avg_Up  = None
        Avg_Down  = None
        RS       = None
        RSI= None
        Signal = None
        # print out
        
        
        # feed df (add new bar / update latest bar)
        df2 = df2.append({"Date": start_time, "Open": first, "High": high, "Low": low, "Close": close, "Volume": volume,	'Complete': complete,	'Price_Diff': Price_Diff,	'Up_Moves': Up_Moves,	'Down_Moves': Down_Moves,	'Avg_Up': Avg_Up,	'Avg_Down': Avg_Down	,'RS': RS	,'RSI': RSI,'Signal': Signal},ignore_index=True)
        df2 = df2.set_index('Date')
        #self.data.loc[start_time] = [first, high, low, close, volume, complete,Price_Diff	,Up_Moves	,Down_Moves	,Avg_Up	,Avg_Down	,RS	,RSI]
        #self.RSI()
        #print(self.data)
        #df2 = df2[df2['Complete']==True]
        self.data = self.data.append(df2)
        self.RSI()
        
        stream_dict = dict(self.data.iloc[-1])
        self.strategy(stream_dict)
        print(self.data)
        
        #stream_dict['Date'] = event_time
        #print("Time: {} | Price: {}| Complete {}| RSI {}| Signal {}".format(event_time, close,complete,stream_dict['RSI'],signal))
        #self.execute_trade(stream_dict,signal)
        
        
    def RSI(self):
        """This function will take historical date and calculate the RSI score

    Args:
        data (data frame): historical data in the form of a pandas data frame
        time_period (_type_): specify trading period to calculate on 
        """
    # https://www.macroption.com/rsi-calculation/ -> rsi calculation
    #RSI = 100 - 100/(1+RS)
    
        #Step 1: calculate up and down moves
        self.data['Price_Diff'] = self.data['Close'].diff(1)
        
        self.data['Up_Moves'] = self.data['Price_Diff'].apply(lambda x: x if x > 0 else 0)
        self.data['Down_Moves'] = self.data['Price_Diff'].apply(lambda x: abs(x) if x < 0 else 0)
        self.data['Avg_Up'] = self.data['Up_Moves'].ewm(span=14).mean()
        self.data['Avg_Down'] = self.data['Down_Moves'].ewm(span=14).mean()
        self.data['RS'] = self.data['Avg_Up']/self.data['Avg_Down'] 
        self.data['RSI'] = self.data['RS'].apply(lambda x: 100-(100/(1+x)))
        #data=data.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
        #self.data = self.data.dropna()
        #self.data = self.data.iloc[1: , :]
        # return data
        
    def strategy(self,stream_data):
        # rsi = stream_data['RSI']
        # if rsi > 70:
        #     return -1
        # elif rsi < 30:
        #     return 1
        # else:
        #     return 0
        buy_price = []
   
        sell_price = []
        rsi_signal = []
        signal = 0
        entry = False
        rsi = [x for x in self.data['RSI']]
        prices = [x for x in self.data['Close']]
        print(len(rsi),len(prices))
        for i in range(len(rsi)):
            
            if rsi[i-1] > 30 and rsi[i] < 30:
                if signal != 1 :
                    buy_price.append(prices[i])
                    sell_price.append(np.nan)
                    signal = 1
                    rsi_signal.append(signal)
                    
                
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    rsi_signal.append(0)
                    #entry = False
            elif rsi[i-1] < 70 and rsi[i] > 70:
                if signal != -1 :
                    buy_price.append(np.nan)
                    sell_price.append(prices[i])
                    signal = -1
                    rsi_signal.append(signal)
                
                
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    rsi_signal.append(0)
                    #entry == True
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
                #entry == False
    
                
                
        # data['Buy Price'] = buy_price
        # data['Sell Price'] = sell_price
        self.data['Signal'] = rsi_signal
            
    def execute_trade(self,stream_data,signal):
        entered_trade = 0
        #print(stream_data)
        print("Time: {} | Price: {}| Complete {}| RSI {}| Signal {}| Entered Trade {}".format(stream_data['Date'], stream_data['Close'],stream_data['Complete'],stream_data['RSI'],stream_data['Signal'],entered_trade))
        if  signal == 1 and stream_data['Complete']==True and entered_trade == 0:
        
                    print('Buying Bitcoin')
                    entered_trade += 1
                    
                    
       
            
        elif  signal == -1 and stream_data['Complete']==True and entered_trade == 1:
                    print('Selling Bitcoin')
                    entered_trade -= 1
                    
                    
        elif signal == 0:
            print('Going neutral')
            
        

In [44]:
trader = LongOnlyTrader('BTCUSDT','1m')

In [45]:
trader.start_trading(2)

2881 2881
                         Open      High       Low     Close    Volume  \
Date                                                                    
2022-03-11 08:14:00  39137.15  39168.09  39125.01  39150.39   2.93052   
2022-03-11 08:15:00  39150.22  39179.03  39150.22  39179.03   3.09587   
2022-03-11 08:16:00  39179.03  39189.57   39179.0  39182.01  2.379471   
2022-03-11 08:17:00   39182.0  39183.23  39146.44  39146.44  3.223509   
2022-03-11 08:18:00  39144.89  39149.75  39084.69  39088.67  1.885879   
...                       ...       ...       ...       ...       ...   
2022-03-13 08:10:00  39071.01  39071.01  39061.89   39065.2  1.552234   
2022-03-13 08:11:00   39065.2  39068.33   39060.0  39068.33  2.858487   
2022-03-13 08:12:00  39068.35  39068.36  39058.96  39058.96  1.748268   
2022-03-13 08:13:00  39060.46  39061.91  39057.88  39057.88  0.358329   
2022-03-13 08:13:00  39060.45  39061.77   39055.0   39055.0   1.19535   

                    Complete Price_Diff 

In [46]:
trader.twm.stop()

2896 2896
                         Open      High       Low     Close    Volume  \
Date                                                                    
2022-03-11 08:14:00  39137.15  39168.09  39125.01  39150.39   2.93052   
2022-03-11 08:15:00  39150.22  39179.03  39150.22  39179.03   3.09587   
2022-03-11 08:16:00  39179.03  39189.57   39179.0  39182.01  2.379471   
2022-03-11 08:17:00   39182.0  39183.23  39146.44  39146.44  3.223509   
2022-03-11 08:18:00  39144.89  39149.75  39084.69  39088.67  1.885879   
...                       ...       ...       ...       ...       ...   
2022-03-13 08:13:00  39060.45  39061.77  39046.05  39051.37  13.14005   
2022-03-13 08:13:00  39060.45  39061.77  39046.05  39051.37  13.14768   
2022-03-13 08:13:00  39060.45  39061.77  39046.05  39051.37  13.54324   
2022-03-13 08:14:00  39051.37  39058.11  39051.36  39058.11   2.04437   
2022-03-13 08:14:00  39051.37  39071.62  39051.36  39071.62    4.7814   

                    Complete Price_Diff 

In [46]:
trader.data.tail(40)

,Open,High,Low,Close,Volume,Complete,Price_Diff,Up_Moves,Down_Moves,Avg_Up,Avg_Down,RS,RSI
Date,,,,,,,,,,,,,
2022-03-11 12:00:00,39894.65,40228.11,39740.43,39906.81,220.179387,True,9.69,9.69,0.00,189.523038,10.164868,18.644910,94.909623
2022-03-11 13:00:00,39909.32,39945.66,39062.5,39260.0,213.934075,True,-646.81,0.00,646.81,156.314119,121.720152,1.284209,56.221170
2022-03-11 14:00:00,39261.1,39595.36,39066.64,39294.37,201.934966,True,34.37,34.37,0.00,135.805728,101.249428,1.341299,57.288662
2022-03-11 15:00:00,39294.37,39299.12,38908.01,39067.08,213.344702,True,-227.29,0.00,227.29,113.735166,121.732999,0.934300,48.301717
2022-03-11 16:00:00,39067.53,39497.1,38592.97,38727.46,208.402869,True,-339.62,0.00,339.62,95.775522,156.139008,0.613399,38.019054
2022-03-11 17:00:00,38727.46,38867.19,38582.82,38815.69,179.087968,True,88.23,88.23,0.00,94.612604,132.074834,0.716356,41.737030
2022-03-11 18:00:00,38815.79,38874.18,38616.39,38632.23,173.581952,True,-183.46,0.00,183.46,80.327827,139.833057,0.574455,36.485967
2022-03-11 19:00:00,38633.77,38865.5,38532.17,38788.35,174.905444,True,156.12,156.12,0.00,91.572610,119.086952,0.768956,43.469477
2022-03-11 20:00:00,38788.35,38962.39,38316.43,38368.33,189.72656,True,-420.02,0.00,420.02,78.187763,163.073279,0.479464,32.407952
